In [1]:
import pandas as pd
import numpy as np

In [2]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
import os

In [ ]:
profile = {'savefile.default_directory': os.getcwd(), 'download.default_directory': os.getcwd()}

chrome_options = webdriver.ChromeOptions()

chrome_options.add_experimental_option('prefs', profile)

In [ ]:
driver = webdriver.Chrome(options=chrome_options) # 크롬드라이버 설치한 경로 작성 필요 
driver.get("https://www.data.go.kr/data/15045048/fileData.do#")

In [ ]:
download_btn = driver.find_element(By.XPATH, '//*[@id="data-url-btn"]')
download_url = download_btn.get_attribute('href')
# driver.to_switch( driver.window_handles[1] )

In [ ]:
driver.get(download_url)
# download_btn.click()

In [ ]:
import zipfile

source_file = '10_42_01_P_CSV.zip'

with zipfile.ZipFile(source_file, 'r') as zf:
    zipinfo = zf.infolist()
    for info in zipinfo:
        info.filename = info.filename.encode('cp437').decode('euc-kr')
        zf.extract(info)

In [3]:
group = pd.read_csv('./fulldata_10_42_01_P_체력단련장업.csv', chunksize = 1000, encoding='cp949')
group = list(group)
df = pd.concat(group)

In [4]:
df_necessary = df.loc[:,['사업장명', '인허가일자', '인허가취소일자','휴업시작일자', '휴업종료일자', '폐업일자', '영업상태명', '소재지전체주소', '도로명전체주소']]

In [5]:
df_necessary.loc[:,'주소'] = np.where(df_necessary.loc[:,'도로명전체주소'].notnull() , df_necessary['도로명전체주소'], df_necessary['소재지전체주소'])

In [6]:
df_necessary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20750 entries, 0 to 20749
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   사업장명     20750 non-null  object 
 1   인허가일자    20750 non-null  int64  
 2   인허가취소일자  17 non-null     float64
 3   휴업시작일자   31 non-null     float64
 4   휴업종료일자   31 non-null     float64
 5   폐업일자     7695 non-null   float64
 6   영업상태명    20750 non-null  object 
 7   소재지전체주소  20393 non-null  object 
 8   도로명전체주소  19973 non-null  object 
 9   주소       20750 non-null  object 
dtypes: float64(4), int64(1), object(5)
memory usage: 1.6+ MB


In [7]:
df_necessary.loc[:,'폐업,전출,휴업,취소,말소여부'] = np.where(df_necessary.loc[:,'인허가취소일자'].notnull() | df_necessary.loc[:,'휴업시작일자'].notnull() | df_necessary.loc[:,'폐업일자'].notnull(), 1, 0)

In [8]:
df_necessary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20750 entries, 0 to 20749
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   사업장명              20750 non-null  object 
 1   인허가일자             20750 non-null  int64  
 2   인허가취소일자           17 non-null     float64
 3   휴업시작일자            31 non-null     float64
 4   휴업종료일자            31 non-null     float64
 5   폐업일자              7695 non-null   float64
 6   영업상태명             20750 non-null  object 
 7   소재지전체주소           20393 non-null  object 
 8   도로명전체주소           19973 non-null  object 
 9   주소                20750 non-null  object 
 10  폐업,전출,휴업,취소,말소여부  20750 non-null  int64  
dtypes: float64(4), int64(2), object(5)
memory usage: 1.7+ MB


In [9]:
df_necessary.loc[:, '폐업,전출,휴업,취소,말소일자'] = df_necessary.loc[:, ['인허가취소일자', '폐업일자', '휴업시작일자']].max(axis=1)

In [10]:
df_necessary.loc[:,'인허가일자'] = pd.to_datetime(df_necessary.loc[:,'인허가일자'], format='%Y%m%d')
df_necessary.loc[:,'인허가취소일자'] = pd.to_datetime(df_necessary.loc[:,'인허가취소일자'],  format='%Y%m%d')
df_necessary.loc[:,'폐업일자'] = pd.to_datetime(df_necessary.loc[:,'폐업일자'],  format='%Y%m%d')
df_necessary.loc[:,'휴업시작일자'] = pd.to_datetime(df_necessary.loc[:,'휴업시작일자'],  format='%Y%m%d')
df_necessary.loc[:,'휴업종료일자'] = pd.to_datetime(df_necessary.loc[:,'휴업종료일자'],  format='%Y%m%d')
df_necessary.loc[:,'폐업,전출,휴업,취소,말소일자'] = pd.to_datetime(df_necessary.loc[:,'폐업,전출,휴업,취소,말소일자'],  format='%Y%m%d')

In [11]:
cols = ['인허가일자', '인허가취소일자','폐업일자','휴업시작일자','휴업종료일자', '폐업,전출,휴업,취소,말소일자',]
for col in cols:
            df_necessary.loc[:,col] = pd.to_datetime(df_necessary.loc[:,col], format= '%Y%m%d')

In [12]:
cols = ['폐업,전출,휴업,취소,말소일자', '인허가일자']
for col in cols:
    df_necessary.loc[:,col[0:-2]+'연월'] = df_necessary.loc[:, col].dt.to_period('M')

In [13]:
df_necessary

,사업장명,인허가일자,인허가취소일자,휴업시작일자,휴업종료일자,폐업일자,영업상태명,소재지전체주소,도로명전체주소,주소,"폐업,전출,휴업,취소,말소여부","폐업,전출,휴업,취소,말소일자","폐업,전출,휴업,취소,말소연월",인허가연월
0,우먼파워에어로빅,2008-09-23,NaT,NaT,NaT,NaT,취소/말소/만료/정지/중지,전라북도 전주시 덕진구 우아동3가 732-5번지,전라북도 전주시 덕진구 백제대로 781 (우아동3가),전라북도 전주시 덕진구 백제대로 781 (우아동3가),0,NaT,NaT,2008-09
1,홍농 헬스클럽,2003-02-12,2020-06-22,NaT,NaT,NaT,취소/말소/만료/정지/중지,전라남도 영광군 홍농읍 상하리 539-1번지,전라남도 영광군 홍농읍 상하길 3,전라남도 영광군 홍농읍 상하길 3,1,2020-06-22,2020-06,2003-02
2,리더스 스포츠클럽,2006-06-12,2020-06-08,NaT,NaT,NaT,취소/말소/만료/정지/중지,전라남도 영광군 영광읍 단주리 630-9번지,전라남도 영광군 영광읍 옥당로 169,전라남도 영광군 영광읍 옥당로 169,1,2020-06-08,2020-06,2006-06
3,태정헬스,2007-11-07,2020-06-08,NaT,NaT,NaT,취소/말소/만료/정지/중지,전라남도 영광군 영광읍 백학리 47번지 6층,전라남도 영광군 영광읍 현암길 45-7,전라남도 영광군 영광읍 현암길 45-7,1,2020-06-08,2020-06,2007-11
4,탑 휘트니스,2004-03-31,2013-12-21,NaT,NaT,NaT,취소/말소/만료/정지/중지,경기도 의왕시 삼동 277-6번지,경기도 의왕시 부곡복지관길 6 (삼동),경기도 의왕시 부곡복지관길 6 (삼동),1,2013-12-21,2013-12,2004-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20745,스파르타짐,2010-01-25,NaT,NaT,NaT,NaT,휴업,경기도 하남시 덕풍동 319-2 남도빌딩,"경기도 하남시 하남대로 837, 남도빌딩 4층 (덕풍동)","경기도 하남시 하남대로 837, 남도빌딩 4층 (덕풍동)",0,NaT,NaT,2010-01
20746,(주)스포츠아트필피트니스,2008-11-04,NaT,2013-05-08,2013-09-01,NaT,휴업,"서울특별시 마포구 상암동 1605번지 누리꿈스퀘어 지층 B1001~1005,1015...","서울특별시 마포구 월드컵북로 396, 지하1층 B1001~1005호 (상암동, 누리...","서울특별시 마포구 월드컵북로 396, 지하1층 B1001~1005호 (상암동, 누리...",1,2013-05-08,2013-05,2008-11
20747,디앤브이 피트니스,2017-05-04,NaT,2019-10-10,2020-10-09,NaT,휴업,서울특별시 강남구 역삼동 783-18번지 제이케이빌딩,"서울특별시 강남구 언주로65길 13, 3층 (역삼동, 제이케이빌딩)","서울특별시 강남구 언주로65길 13, 3층 (역삼동, 제이케이빌딩)",1,2019-10-10,2019-10,2017-05
20748,대영청소년문화진흥원(주),2009-02-24,NaT,NaT,NaT,NaT,영업/정상,전라남도 화순군 북면 옥리 499-3번지,전라남도 화순군 북면 옥리길 27-20,전라남도 화순군 북면 옥리길 27-20,0,NaT,NaT,2009-02


In [14]:
df_necessary.loc[:,'폐업,전출,휴업,취소,말소연월'] = df_necessary.loc[:,'폐업,전출,휴업,취소,말소일자'].dt.to_period('M')

In [26]:
df_necessary.loc[:,'인허가연월'] = df_necessary.loc[:,'인허가일자'].dt.to_period('Y')

In [27]:
df_necessary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20750 entries, 0 to 20749
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   사업장명              20750 non-null  object        
 1   인허가일자             20750 non-null  datetime64[ns]
 2   인허가취소일자           17 non-null     datetime64[ns]
 3   휴업시작일자            31 non-null     datetime64[ns]
 4   휴업종료일자            31 non-null     datetime64[ns]
 5   폐업일자              7695 non-null   datetime64[ns]
 6   영업상태명             20750 non-null  object        
 7   소재지전체주소           20393 non-null  object        
 8   도로명전체주소           19973 non-null  object        
 9   주소                20750 non-null  object        
 10  폐업,전출,휴업,취소,말소여부  20750 non-null  int64         
 11  폐업,전출,휴업,취소,말소일자  7743 non-null   datetime64[ns]
 12  폐업,전출,휴업,취소,말소연월  7743 non-null   period[M]     
 13  인허가연월             20750 non-null  period[A-DEC] 
 14  xx                0 no

In [28]:
regions = {"서울특별시":"Seoul", "부산광역시": "Busan", "인천광역시": "Incheon", "대구광역시": "Daegu", "대전광역시": "Daejeon", "광주광역시": "Gwangju", "울산광역시": "Ulsan", "세종특별자치시": "Sejong", "경기도": "Gyeonggi", "강원도": "Gangwon", "충청북도": "Chungbuk", "충청남도": "Chungnam", "전라북도": "Jeonbuk", "전라남도": "Jeonnam", "경상북도": "Gyeongbuk", "경상남도":"Gyeongnam", "제주특별자치도": "Jeju"}

In [29]:
for region in regions.keys():
    globals()['df_{}'.format(regions[region])] = df_necessary[(df_necessary['주소'].str.contains(region, na = False))]


In [30]:
tot = 0
for i in regions.keys():
    tot += len(globals()['df_{}'.format(regions[i])])
tot

20750

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rc

rc('font', family='AppleGothic')

df_necessary.groupby(df_necessary.loc[:,'인허가연월'])['인허가연월'].count().plot(color = 'r', linewidth = '1.5')
df_necessary.groupby(df_necessary.loc[:,'폐업,전출,휴업,취소,말소연월'])['폐업,전출,휴업,취소,말소여부'].sum().plot(c = '#4CAF50', linewidth = '0.8')
plt.xlim('1981-01', '2022-10')
plt.xlabel("월")
plt.ylabel("업장 수")
plt.title("연도 및 월별 체력단련장업 추이")
plt.show()